[View in Colaboratory](https://colab.research.google.com/github/shalini0289/Colab/blob/master/lstm.ipynb)

In [35]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9458586611772208111]

In [0]:
from google.colab import auth
from googleapiclient.discovery import build

In [0]:
import io , requests
import sys

import pandas as pd
import os
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
SOURCE_FOLDER=''

In [0]:
def get_parent_folder(folder_name):
  page_token = None
  folder_array = []
  query = "name='%s' and mimeType='application/vnd.google-apps.folder'" % folder_name
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.append({"name" : file.get('name'), "id" : file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

In [0]:
def get_files_from_parent(parent_id):
  page_token = None
  folder_array = dict()
  query = "'%s' in parents" % parent_id
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.update({file.get('name'):file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

In [0]:
def get_file_buffer(file_id, verbose=0):
  from googleapiclient.http import MediaIoBaseDownload
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    progress, done = downloader.next_chunk()
    if verbose:
      sys.stdout.flush()
      sys.stdout.write('\r')
      percentage_done = progress.resumable_progress * 100/progress.total_size
      sys.stdout.write("[%-100s] %d%%" % ('='*int(percentage_done), int(percentage_done)))
  downloaded.seek(0)
  return downloaded

In [43]:
parent_folder = get_parent_folder('Kaggle_comp')
print(parent_folder)
parent_folder[0]["id"]

[{'name': 'kaggle_comp', 'id': '1X5a6lJ9jtz4tYlo5eq2dxIkeN0kjgPIU'}]


'1X5a6lJ9jtz4tYlo5eq2dxIkeN0kjgPIU'

In [44]:
input_file_meta = get_files_from_parent(parent_folder[0]["id"])
print(input_file_meta)

{'Sentiment.csv': '1okO2cpvnKfNPNBLo5pgb58LkcVdxvH1m', 'glove.6B.50d.txt': '1hUQsY4XPTXsEN4PpeLAXR93uKMUt6THA'}


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [46]:
for file, id in input_file_meta.items():
  downloaded = get_file_buffer(id, verbose=1)
  dest_file = os.path.join(SOURCE_FOLDER, file)
  print("processing %s data" % file)
  with open(dest_file, "wb") as out:
    out.write(downloaded.read())
    print("Done %s" % dest_file)

[====================================================================================================] 100%processing Sentiment.csv data
Done Sentiment.csv
[====================================================================================================] 100%processing glove.6B.50d.txt data
Done glove.6B.50d.txt


In [0]:
data=pd.read_csv(os.path.join(SOURCE_FOLDER,'Sentiment.csv'))

data = data[['text','sentiment']]

In [50]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [51]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [58]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
 - 20s - loss: 0.1735 - acc: 0.9297
Epoch 2/20
 - 19s - loss: 0.1620 - acc: 0.9350
Epoch 3/20
 - 21s - loss: 0.1485 - acc: 0.9430
Epoch 4/20
 - 20s - loss: 0.1369 - acc: 0.9449
Epoch 5/20
 - 21s - loss: 0.1254 - acc: 0.9482
Epoch 6/20
 - 20s - loss: 0.1222 - acc: 0.9494
Epoch 7/20
 - 20s - loss: 0.1126 - acc: 0.9519
Epoch 8/20
 - 21s - loss: 0.1110 - acc: 0.9560
Epoch 9/20
 - 20s - loss: 0.1038 - acc: 0.9583
Epoch 10/20
 - 21s - loss: 0.0953 - acc: 0.9590
Epoch 11/20
 - 20s - loss: 0.0915 - acc: 0.9622
Epoch 12/20
 - 20s - loss: 0.0913 - acc: 0.9602
Epoch 13/20
 - 20s - loss: 0.0930 - acc: 0.9602
Epoch 14/20
 - 20s - loss: 0.0872 - acc: 0.9638
Epoch 15/20
 - 20s - loss: 0.0858 - acc: 0.9662
Epoch 16/20
 - 20s - loss: 0.0867 - acc: 0.9623
Epoch 17/20
 - 20s - loss: 0.0811 - acc: 0.9681
Epoch 18/20
 - 20s - loss: 0.0760 - acc: 0.9677
Epoch 19/20
 - 20s - loss: 0.0805 - acc: 0.9661
Epoch 20/20
 - 20s - loss: 0.0728 - acc: 0.9694


In [0]:
model.save(os.path.join(SOURCE_FOLDER,'lstm.h5'))


In [60]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.32
acc: 0.80


In [62]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 51.369863013698634 %
neg_acc 90.56291390728477 %


In [63]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  206  633    6  150    5   55 1055   55   46    6  150]]
negative
